## **Uploading and reading the dataset**

In [ ]:
from google.colab import files 
uploaded = files.upload()

### **Run this cell of code only for the first time!**

In [ ]:
!pip install colored

### **Read the Dataset and provide initial information about it**

In [ ]:
import numpy as np
import pandas as pd
import colored


# COMMENT OUT AND IN THE DATASET OF THE PROJECT TO BE ANALYZED 

df = pd.read_csv("xd_cleaned.csv")
#df = pd.read_csv("apstud_cleaned.csv")
#df = pd.read_csv("tistud_cleaned.csv")
#df = pd.read_csv("mobile_cleaned.csv")
#df = pd.read_csv("mdl_cleaned.csv")
#df = pd.read_csv("dnn_cleaned.csv")
#df = pd.read_csv("mesos_cleaned.csv")
#df = pd.read_csv("mule_cleaned.csv")
#df = pd.read_csv("nexus_cleaned.csv")
#df = pd.read_csv("timob_cleaned.csv")

pd.set_option('display.max_rows', df.shape[0]+1)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 150)

print("\n******************************\n")
print("Printing Dataset Initial Shape: \n")
print(df.shape)
print("\n******************************\n")
print("Printing Dataset Initial Descriptives: \n")
print(df.describe())
print("\n******************************\n")
print("Printing Dataset Information: \n")
print(df.info())
print("\nCheck the data fields that contain missing values in the Open-Source Project \n")
print(df.isnull().sum())

### **Read the Changeset Dataset and provide initial information about it**

In [ ]:
import numpy as np
import pandas as pd
import colored


# COMMENT OUT AND IN THE CHANGELOG DATASET OF THE PROJECT TO BE ANALYZED 

df_changelog = pd.read_csv("xd_dataframe.csv")
#df_changelog = pd.read_csv("apstud_cleaned.csv")
#df_changelog = pd.read_csv("tistud_cleaned.csv")
#df_changelog = pd.read_csv("mobile_cleaned.csv")
#df_changelog = pd.read_csv("mdl_cleaned.csv")
#df_changelog = pd.read_csv("dnn_cleaned.csv")
#df_changelog = pd.read_csv("mesos_cleaned.csv")
#df_changelog = pd.read_csv("mule_cleaned.csv")
#df_changelog = pd.read_csv("nexus_cleaned.csv")
#df_changelog = pd.read_csv("timob_cleaned.csv")

pd.set_option('display.max_rows', df_changelog.shape[0]+1)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 150)

print("\n******************************\n")
print("Printing Dataset Initial Shape: \n")
print(df_changelog.shape)
print("\n******************************\n")
print("Printing Dataset Initial Descriptives: \n")
print(df_changelog.describe())
print("\n******************************\n")
print("Printing Dataset Information: \n")
print(df_changelog.info())
print("\nCheck the data fields that contain missing values in the Open-Source Project \n")
print(df_changelog.isnull().sum())

### **Insights on total Number of Issues, Sprints and Developers before running the rules:**

In [ ]:
print("Number of issues: \n")
print(df.key.nunique())
print(df.key.unique())
print('\n\n')
print(len(df.key.unique()))

In [ ]:
print("Number of sprints: \n")
print(df.sprint.nunique())
print(df.sprint.unique())
print('\n\n')
print(len(df.sprint.unique()))

In [ ]:
print("Number of developers: \n")
print(df['assignee.name'].nunique())
print(df['assignee.name'].unique())
print('\n\n')
print(len(df['assignee.name'].unique()))

## **#Rules for tracking issue statuses!**

In [ ]:
df_changelog.shape

In [ ]:
df_changelog.dropna(subset = ["from"], inplace=True)
df_changelog.dropna(subset = ["to"], inplace=True)

In [ ]:
df_changelog = df_changelog[~df_changelog['from'].str.contains("[a-zA-Z]").fillna(False)]
df_changelog = df_changelog[~df_changelog['to'].str.contains("[a-zA-Z]").fillna(False)]

In [ ]:
df_changelog['from'] = pd.to_numeric(df_changelog['from'], errors='coerce')
df_changelog['to'] = pd.to_numeric(df_changelog['to'], errors='coerce')

In [ ]:
df_changelog.dtypes

In [ ]:
df_changelog.shape

In [ ]:
df_changelog['from'].value_counts()

In [ ]:
statuses_to_keep = [3, 10000, 10001, 10006]

df_changelog = df_changelog[df_changelog['to'].isin(statuses_to_keep)]
df_changelog = df_changelog[df_changelog['from'].isin(statuses_to_keep)]

In [ ]:
df_changelog = df_changelog.sort_values(['key', 'created'], ascending=[True, True])

grouped_df = df_changelog.groupby("key")

for key, item in grouped_df:
    print(grouped_df[['key', 'from', 'to']].get_group(key).to_markdown())
   
   
   
   
   
    # if (pd.isnull(grouped_df['key'])):
    #   print('\n\n*        FALSEEE1        *\n\n')
    #   #exit()
    # else:
    #   print('\n\n*        TRUE        *\n\n')
    #   print('\n\n*        NEXT SPRINT        *\n\n')

In [ ]:
df_changelog = df_changelog.sort_values(['key', 'created'], ascending=[True, True])

df_changelog = df_changelog.set_index('key')

print (df_changelog.index[df_changelog['from']].tolist())
print (df_changelog.index[df_changelog['to']].tolist())

In [ ]:
df['status.name'].unique()

In [ ]:
df['status.id'].unique()

In [ ]:
df[['key','status.name', 'status.id']].head(50)

# **Scrum Rules Implementation**

## **Data Pre-processing!**

## **#R1 - No more than five weeks should elapse for a single sprint.**

### **Code for Mobile and MDL Projects!**

In [ ]:
# REMOVE ISSUES WITH EMPTY SPRINT VALUE
df = df[df['sprint'].notna()]

df['sprint_start_date'] = df['sprint'].astype(str).str.extract('startDate=(.{,24})')
df['sprint_end_date'] = df['sprint'].astype(str).str.extract('endDate=(.{,24})')
df["sprint_id"] = df["sprint"].str.findall(r"id\=(\d+),")
df['sprint_id'] = df['sprint_id'].str.join(',')

# THE LINE OF CODE BELOW BELONGS ONLY TO MDL PROJECT 
#df = df.drop(df[df['sprint_start_date']== '<null>,endDate=<null>,co'].index)

df['sprint_start_date'] = pd.to_datetime(df['sprint_start_date'])
df['sprint_end_date'] = pd.to_datetime(df['sprint_end_date'])

df.sprint_id.value_counts()

In [ ]:
len(df.sprint.value_counts())

In [ ]:
df["sprint_id"] = df["sprint_id"].str.replace(",","invalid")
df = df[~df.sprint_id.str.contains('invalid')]

df.sprint_id = pd.to_numeric(df.sprint_id, errors='coerce')
df = df.sort_values(by=['sprint'])
df['difference_between_sprints_in_days'] = df['sprint_end_date'] - df['sprint_start_date']

grouped_df = df.sort_values('sprint_start_date').groupby(["sprint_id"])
#display(df[['sprint_id', 'sprint_start_date', 'sprint_end_date', 'difference_between_sprints_in_days']])
first_values = grouped_df.first()
display(first_values[['sprint_start_date', 'sprint_end_date', 'difference_between_sprints_in_days']])

In [ ]:
len(df.sprint_id.unique())

In [ ]:
df.sprint_id.unique()

sprints = [125, 106, 167, 289, 264, 195, 114, 136,  85, 170, 165,  79,  92, 160, 143, 202, 176,  75, 229,  95,  74, 152]
sprints.sort()
df = df[df['sprint_id'].isin(sprints)]

df[['sprint_id', 'difference_between_sprints_in_days']]

In [ ]:
df['difference_between_sprints_in_days'].value_counts()

#df.sprint.nunique()

In [ ]:
df['difference_between_sprints_in_days'].describe()

In [ ]:
df['sprint_id'].value_counts()

In [ ]:
df['difference_between_sprints_in_days'].describe()

### **Code for Mule Project!**

In [ ]:
df['sprint'].value_counts()

In [ ]:
# remove empty sprints
df = df[df['sprint'].notna()]

In [ ]:
df['sprint'] = df['sprint'].str.replace('startDate=', '')
df['sprint'] = df['sprint'].astype(object).replace('<null>', np.nan)

df['sprint']= pd.to_datetime(df['sprint'])

In [ ]:
len(df['sprint'].unique())

In [ ]:
df['sprint'].value_counts()

In [ ]:
df_sorted = df.sort_values(by=['sprint'])
df_sorted['difference'] = (df_sorted['sprint'] - df_sorted['sprint'].shift(1)).dt.days
print(df_sorted.difference.describe())
df_sorted = df_sorted.groupby('sprint')
first_values = df_sorted.first()
display(first_values[['difference']])

In [ ]:
df_sorted['difference'].value_counts()

In [ ]:
# df['count_difference'] = (df['sprint'] - df['sprint'].shift(1)).dt.days 

# df[df['count_difference'] > 28.0 ].count() 

In [ ]:
df_sorted.difference.describe()

## **#R4 - The duration of all sprints should follow similar pace.**

## **#This Rule is checked and reported through R1!**

## **#R5 - The next Sprint execution should begin only after the previous Sprint's resolution.**

In [ ]:
df = df[df['sprint'].notna()]

df['sprint_start_date'] = df['sprint'].astype(str).str.extract('startDate=(.{,24})')
df['sprint_end_date'] = df['sprint'].astype(str).str.extract('endDate=(.{,24})')

df["sprint_id"] = df["sprint"].str.findall(r"id\=(\d+),")
df['sprint_id'] = df['sprint_id'].str.join(',')

df = df.drop(df[df['sprint_start_date']== '<null>,endDate=<null>,co'].index)

df['sprint_start_date'] = pd.to_datetime(df['sprint_start_date'])
df['sprint_end_date'] = pd.to_datetime(df['sprint_end_date'])

df['day_of_creation'] = df.sprint_start_date.dt.dayofyear
df['day_of_completion'] = df.sprint_end_date.dt.dayofyear 

#df

In [ ]:
final_df = df.sort_values(by=['sprint_start_date'], ascending=True)

final_df.head(20)

In [ ]:
# Run only for MDL

df["sprint_id"] = df["sprint"].str.findall(r"id\=(\d+),")
df['sprint_id'] = df['sprint_id'].str.join(',')

In [ ]:
df["sprint_id"] = df["sprint_id"].str.replace(",","invalid")
df = df[~df.sprint_id.str.contains('invalid')]

df.sprint_id = pd.to_numeric(df.sprint_id, errors='coerce')

In [ ]:
final_df = df.sort_values(by=['sprint_start_date'], ascending=True)

final_df['difference'] = (final_df['sprint_start_date'] - final_df['sprint_end_date'].shift(1)).dt.days
final_df.head(20)

#df = df.sort_values(by=['sprint_id'])

grouped_df = final_df.groupby("sprint_id")
first_values = grouped_df.first()
display(first_values[[ 'sprint_start_date', 'sprint_end_date', 'day_of_creation', 'day_of_completion',  'difference']])

In [ ]:
first_values.difference.value_counts()

In [ ]:
first_values.difference.describe()

## **R6 - There should be a project clarity identifier attached to each issue.**

In [ ]:
# THIS RULE CHECKS IF ALL ISSUES IN THE PROJECT, WITHIN THEIR CORRESPONDING SPRINTS
# HAVE a PROJECT Clarity Identifier assosciated with them, which indicates that the issue belongs to a valid project.

print(f'Total number of issues present in this Open-Source Project is: {df.key.nunique()}.')
print(f'Total number of missing issues present in this Open-Source Project is: {df.key.isnull().sum()}.\n')
print(f'Total number of sprints present in this Open-Source Project is: {df.sprint.nunique()}.')
print(f'Total number of missing sprint IDs present in this Open-Source Project is: {df.sprint.isnull().sum()}.\n')

grouped_df = df.groupby("sprint")

print('\n\n*      FIRST SPRINT       *\n\n')
for key, item in grouped_df:
  if item.iloc[0]['project'] != item['project'].iloc[0]:
    print("False")
  else: 
    print("True")
    # print(grouped_df[['key', 'project']].get_group(key).to_markdown())
    #print('\n\n*      NEXT SPRINT       *\n\n')
    
r7_df = df[(df['project'].isnull()) & pd.notnull(df["key"])]

if (len(r7_df[['key', 'sprint']]) == 0) :
  print(colored.fg("green") + "\nRule 7 passed for this Open-Source Project!\n")
  print(f'Total number of missing project clarity IDs present in this Open-Source Project is: {df.project.isnull().sum()}.\n')
else :
  print(colored.fg("red") + "\nRule 7 failed for this Open-Source Project!\n")

r7_df[['sprint', 'key']]

## **#R7 - No considerable amount of time should elapse between the finish of a sprint and the beginning of the new sprint.**

## **This rule is checked through R1!**

## **#R8 - There should not be a considerable amount of time for a developer to volunteer and start a new issue after she/he has completed the previous one.**

## **This rule should be checked AFTER #R28 has been checked!**

In [ ]:
# rename the column: assignee.name, to have the new name: developers
#df.rename(columns={'assignee.name': 'developers'}, inplace=True)

#df.head()

MAX_DAY = 2.0
counter = 0

# For each project, take the list of active developers, and then remove all other developers that are not active part of the team!

# print("for XD")
#ACTIVE_DEVS = ['grussell', 'dturanski', 'iperumal', 'hillert', 'eric.bottard', 'thomas.risberg', 'grenfro']

# print("for APSTUD")
#ACTIVE_DEVS = ['cwilliams', 'pinnamuri', 'mxia', 'sgibly']

# print("for TISTUD")
#ACTIVE_DEVS = ['pinnamuri', 'sgibly', 'mxia', 'cwilliams', 'kkolipaka']

# print("for MOBILE")
#ACTIVE_DEVS = ['dpalou', 'jleyva', 'pferre22']

# print("for MDL")
#ACTIVE_DEVS = ['timhunt', 'mudrd8mz', 'dobedobedoh', 'danmarsden', 'jleyva', 'stronk7', 'marina', 'dmonllao', 'fred', 'quen', 'markn', 'damyon', 'dougiamas', 'moodle.com', 'ankit_frenz', 'poltawski', 'rajeshtaneja', 'andyjdavis', 'skodak', 'samhemelryk', 'jerome', 'dongsheng', 'lazyfish']

# print("for DNN")
#ACTIVE_DEVS = ['bing.wu', 'robert.cui', 'KenGrierson', 'Amritpal.Manak', 'mohit', 'behzad.basir']

# print("for MESOS")
#ACTIVE_DEVS = ['js84', 'jieyu', 'kaysoky', 'jojy', 'anandmazumdar', 'jvanremoortere', 'greggomann', 'gilbert', 'vinodkone', 'alexr', 'mcypark', 'karya', 'bmahler']

# print("for MULE")
#ACTIVE_DEVS = ['mariano.gonzalez', 'rodrigo.merino', 'afelisatti', 'pablo.lagreca.ce', 'pablo.kraan', 'andres.gregoire', 'marcosnc']

# print("for NEXUS")
#ACTIVE_DEVS = ['jtom', 'cstamas', 'alin', 'plynch']

# print("for TIMOB")
#ACTIVE_DEVS = ['kota', 'cwilliams', 'msamah', 'gmathews', 'hansknoechel', 'cng', 'penrique', 'hpham', 'cbarber', 'vduggal']

df = df[df['developers'].isin(ACTIVE_DEVS)]
print(df.shape)


# Remove issues with no end date, and issues belonging to no sprints!
df = df[df['resolutiondate'].notna()]
df = df[df['sprint'].notna()]
df = df[df['developers'].notna()]
#df.shape

# Extract days from created and resolutiondate
df['created'] = pd.to_datetime(df.created, utc=True)
df['resolutiondate'] = pd.to_datetime(df.resolutiondate, utc=True)
df['day_of_creation'] = df.created.dt.dayofyear
df['day_of_completion'] = df.resolutiondate.dt.dayofyear 
df['issue_completion_time'] =  df.resolutiondate - df.created

df.head()

grouped_df = df.sort_values(['sprint', 'created', 'key']).groupby(["sprint", "developers"])

# grouped_df.sort_values(by=['sprint', 'created', 'priority.name'])
print('\n\n*      FIRST SPRINT       *\n\n')
for key, item in grouped_df:
    print(f"Key is: {key}")
    #item.sort_index()
    item['diff'] = (item['created'] - item['resolutiondate'].shift(1)).dt.days 
    print(f"{item[['key', 'developers', 'issue_completion_time', 'diff']].to_markdown()}")

## **#R24 - The backlog does not contain meaningless or empty issues. I consider meaningless issues the issues that belong to no project and have no issue body, description, start and end time and other details that are relevant to developers and other roles.**

In [ ]:
# A meaningless issue corresponds to an issue which does not have any description and/or summary. 
# Description field is more important in this rule.  
# Moreover, we can check in this rule whether all issues have any missing status names, priority labels and story points.

In [ ]:
# rename the column: status.name, to have the new name: status
df.rename(columns={'status.name': 'status'}, inplace=True)
#df.head()

for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

In [ ]:
cols = ['description', 'summary', 'status', 'project', 'storypoints', 'priority.name']
#df.sprint = pd.to_numeric(df.sprint, errors='coerce')
df = df.sort_values(by="sprint", ascending=False)
print("\n      CHECKING IF THERE ARE MISSING INFORMATION IN THE DESCRITPIVE FIELDS OF THE ISSUES \n\n")
df.set_index('sprint')[cols].isna().sum(level=0)

In [ ]:
meaningless_issues_per_sprint = df.groupby('sprint').description.nunique()
pd.set_option('display.max_rows', df.shape[0]+1)
meaningless_issues_per_sprint

In [ ]:
df.isnull().sum()

In [ ]:
df.sprint.value_counts()

In [ ]:
no_end_date_df = df[['key', 'sprint', 'priority.name']][df['priority.name'].isna()]
no_end_date_df.sort_values(by="sprint")

In [ ]:
no_end_date_df = df[['key', 'sprint', 'priority.name', 'project', 'description']][df['storypoints'].isna()]
no_end_date_df.sort_values(by="sprint")

In [ ]:
no_end_date_df = df[['key', 'sprint']][df['resolutiondate'].isna()]
no_end_date_df.sort_values(by="sprint")

## **#R27 - "No more than 8 active developers should be involved in development tasks"**

In [ ]:
# rename the column: assignee.name, to have the new name: developers
df.rename(columns={'assignee.name': 'developers'}, inplace=True)

df.head(2)

In [ ]:
print(df.key.count())
print('\n\n')
print(df.isnull().sum())

print(f"\n\nOut of {df.key.count()} issues, {df.key.count()-df.developers.isnull().sum()} of them have developers assigned to them.")

In [ ]:
# Checking how many unique sprints are there (excluding if there is nan values):

print("\nCheck the total number of sprints in the Open-Source Project \n")

print(df.sprint.nunique())

In [ ]:
# Checking the most active developers
print(df.developers.describe())
print('\n\n')
print(df.developers.value_counts())
print(df.developers.value_counts().mean())

In [ ]:
value_counts = df['developers'].value_counts()

#to_remove = value_counts[value_counts <= df.developers.value_counts().mean()].index

#print('for xd')  
#to_remove = value_counts[value_counts <= 98].index

#print('for apstud')
#to_remove = value_counts[value_counts <= 59].index

#print('tistud')
#to_remove = value_counts[value_counts <= 100].index

#print('mobile')
#to_remove = value_counts[value_counts <= 100].index

#mdl
#to_remove = value_counts[value_counts <= 386].index

#dnn
#to_remove = value_counts[value_counts <= 62].index

#mesos
#to_remove = value_counts[value_counts <= 31].index

#mule
#to_remove = value_counts[value_counts <= 73].index

#nexus
#to_remove = value_counts[value_counts <= 56].index

#timob  
#to_remove = value_counts[value_counts <= 73].index

# Keep rows where the developers column is not in to_remove
df = df[~df.developers.isin(to_remove)]

devs_per_sprint = df.groupby('sprint').developers.nunique()

#pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option('display.max_rows', df.shape[0]+1)

devs_per_sprint.sort_values(ascending=False)
#print(devs_per_sprint.count())

In [ ]:
# Checking how many unique issues are there after removing non-active developers:

print("\nCheck the total number of sprints in the Open-Source Project \n")

print(df.key.nunique())

In [ ]:
# Checking how many unique sprints are there after removing non-active developers:

print("\nCheck the total number of sprints in the Open-Source Project \n")

print(df.sprint.nunique())

In [ ]:
print("Number of active developers: \n")
print(df['developers'].nunique())
print(df['developers'].unique())
print('\n\n')
print(len(df['developers'].unique()))

## **#R28 - The status of issues should always follow the agreed workflow, depending on the project and development team. (#TODO)**

In [ ]:
print(df['status.name'].value_counts())
#df.groupby('key')
#df[['sprint', 'key', 'status.id',	'status.name']].head(20)
print(df.project[0])

## **R30: No previously resolved issue should reappear in a future sprint. (#TODO)**

In [ ]:
#TODO

## **#R30 - Higher priority issues should have an earlier resolution date than low priority issues, if created at the same time. (not feasible - delted)**

### **ALGORITHM:**

group issues by sprint

order issues by end_date

if(issue['priority'] == null):

  discard issue

else:

  print(issue['start_date', 'end_date', 'priority'])

for each pair of consecutive issue (issue_1, issue_2):

  if(issue1.priority == 'major' AND issue_2.priority !== 'major' ):

  if(issue1.end_date < issue2.end_date):

  FALSE

  else

  TRUE

In [ ]:
df.shape

In [ ]:
#Converting the dates to pandas datetime format, which will be easier to process.

df['created'] = pd.to_datetime(df.created)
df['resolutiondate'] = pd.to_datetime(df.resolutiondate)
df.dtypes
df['day_of_creation'] = df.created.dt.dayofyear
df['day_of_completion'] = df.resolutiondate.dt.dayofyear 

df['issue_completion_time'] =  df.resolutiondate - df.created
df['issue_completion_time'].value_counts()




In [ ]:
#https://stackoverflow.com/questions/54244171/how-do-i-loop-over-each-row-in-a-pandas-groupby

# drop issues with no end date
df = df[df['resolutiondate'].notna()]
#df.shape


# extract days from created and resolutiondate

df['created'] = pd.to_datetime(df.created)
df['resolutiondate'] = pd.to_datetime(df.resolutiondate)
df['day_of_creation'] = df.created.dt.dayofyear
df['day_of_completion'] = df.resolutiondate.dt.dayofyear 
df['issue_completion_time'] =  df.resolutiondate - df.created

df.head()


grouped_issues = df.groupby('sprint').nunique()
pd.set_option('display.max_rows', df.shape[0]+1)
#display(grouped_issues.key)
grouped_issues
#print(df.columns)
issues_df = df[['sprint', 'key', 'priority.name', 'created', 'resolutiondate', 'issue_completion_time', 'day_of_creation', 'day_of_completion']]
# priorities_df = issues_df['priority.name'].unique()
# print(priorities_df)
print('\n\n')
issues_df.sort_values(by=['sprint', 'day_of_creation', 'priority.name'])

In [ ]:
df['created'] = pd.to_datetime(df.created)
df['day_of_creation'] = df.created.dt.dayofyear 
df.created = df.created.dt.strftime('%Y-%m-%d')
#grouped_issues = df.groupby(['sprint', 'day_of_creation'])

issues_df = df[['key', 'priority.name', 'created', 'resolutiondate', 'sprint', 'day_of_creation']]
#issues_df.groupby(['sprint'])
issues_df.sort_values(by=['sprint', 'created', 'priority.name', 'resolutiondate'])
issues_df

In [ ]:
# grouped_df = issues_df.groupby("sprint")

# for key, item in grouped_df:
#     print(grouped_df[['sprint', 'key', 'priority.name', 'created', 'resolutiondate', 'issue_completion_time', 'day_of_creation', 'day_of_completion']].get_group(key).to_markdown())
#     #if (grouped_df['day_of_creation'].iloc[key] == grouped_df['day_of_creation'].iloc[key+1]):
#         if (grouped_df['priority.name'].iloc[key] != grouped_df['priority.name'].iloc[key+1]):
#           print('\n\n*        FALSEEE1        *\n\n')
#       #exit()
#     else:
#       print('\n\n*        TRUE        *\n\n')
#       print('\n\n*        NEXT SPRINT        *\n\n')


      

In [ ]:
df.groupby('priority.name').filter(lambda x: x.created.nunique()>1)

In [ ]:
for (columnName, columnData) in df.iteritems():
  if(columnName )
   print('Colunm Name : ', columnName)
   print('Column Contents : ', columnData.values)

In [ ]:
#column = 'sprint'
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

In [ ]:
# drop issues with no end date
df = df[df['resolutiondate'].notna()]
df = df[df['sprint'].notna()]
#df.shape

# extract days from created and resolutiondate

df['created'] = pd.to_datetime(df.created)
df['resolutiondate'] = pd.to_datetime(df.resolutiondate)
df['day_of_creation'] = df.created.dt.dayofyear
df['day_of_completion'] = df.resolutiondate.dt.dayofyear 
df['issue_completion_time'] =  df.resolutiondate - df.created

df.head()

In [ ]:
duplicateRowsDF = df[df.duplicated(['day_of_creation'])]


#duplicateRowsDF[['sprint', 'key', 'day_of_creation']].head(20)

duplicateRowsDF = duplicateRowsDF[['sprint', 'key', 'priority.name', 'issue_completion_time', 'day_of_creation', 'day_of_completion']]
# priorities_df = issues_df['priority.name'].unique()
# print(priorities_df)
print('\n\n')
duplicateRowsDF.sort_values(by=['sprint', 'day_of_creation', 'priority.name'])

In [ ]:
df.sort_values(by=[ 'day_of_creation', 'priority.name'])
grouped_df = df.groupby(["sprint"])

#grouped_df.sort_values(by=['sprint', 'created', 'priority.name'])

print('\n\n*      FIRST SPRINT       *\n\n')
for key, item in grouped_df:
    print(f"Key is: {key}")
    item.sort_values('created')
    print(f"{item[['sprint', 'key','priority.name', 'day_of_creation', 'day_of_completion', 'issue_completion_time']].to_markdown()}")
    
    #print(grouped_df[['sprint', 'key', 'priority.name', 'day_of_creation', 'day_of_completion', 'issue_completion_time']].get_group(key).to_markdown())
    

## **#R31 - Each Scrum Sprints can be considered a short project, therefore there should be a unique identifier/name associated with each Sprint**

In [ ]:
# THIS RULE CHECKS IF ALL EXISTING SPRINTS ARE UNIQUELY IDENTIFIED, NOT IF THERE ARE ISSUES NOT BELONGING TO A SPRINT!! 

#print(f"\nTotal records of Sprints in the Open-Source Project: {df['sprint'].count()}\n")

print(f"Number of unique sprints (excluding the nan) in the Open-Source Project: {df.sprint.nunique()}\n")
#print(df.sprint.unique())

print(f"The number of missing values in the Sprint field in the Open-Source Project is: {df.sprint.isnull().sum()}\n")
print(f"The number of non-missing values in the Sprint field in the Open-Source Project is: {df.sprint.notnull().sum()}\n")
list_of_sprints = df['sprint'].unique()
print(f'There are {len(list_of_sprints)} unique sprints in this project!\n')

if (df.sprint.nunique() == len(list_of_sprints)-1):
  print(colored.fg("green") + f"Rule 33 Passes for project: {df.project[0]}!")
else:
  print(colored.fg("red") + f"Rule 33 Fails for project: {df.project[0]} !")

#sprint_df = df[df["sprint"].notna()]
#sprint_df.sprint.drop_duplicates().sort_values(ascending=True)

In [ ]:
display(df.sprint.value_counts())

In [ ]:
# column = ['sprint']

# for (columnName, columnData) in df.iteritems():
#   if columnName:
#     print('Column Name : ', columnName)
#     print('Column Contents : ', columnData.values)

# for name in df.columns:
#   for row in df.index:
#       if df.loc[name] == 'sprint':
#           print("sprint")
            #df.loc[row,name] = name

In [ ]:
spike_cols = [col for col in df.columns if 'sprint' in col]
print(list(df.columns))
print(spike_cols)

In [ ]:
df['empty_sprint_IDs'] = df['sprint'].apply(lambda x: 'True' if pd.isnull(x) else 'False')

df['empty_sprint_IDs'].value_counts()

## **#R32 - There should be a type, such as bug, improvement or task, associated to each issue**

In [ ]:
# THIS RULE CHECKS IF ALL ISSUES IN THE PROJECT, ARE ASSIGNED AN ISSUE STATUS (IN PROGRESS, IN TEST, ETC.)
# NOTE: Each project has its own types, this rule only checks if issues have types assigned to them throughout their lifecycle. 


df.rename(columns={'issuetype.name': 'types'}, inplace=True)

print(f'Total number of issue types present in this Open-Source Project is: {df.types.nunique()}.')
print('\nThese different statuses and their number of occurence for this Open-Source Project are:')
print(df['types'].value_counts())

print('\n\nFinally, we are checking if there are issues having no issue types in the dataset: \n')
types_df = df[(df['types'].isnull()) & pd.notnull(df["key"])]

print(f"There are {len(df['types'])} values present in the issuetype.name data field.")
print(f"Total number of issues having no issue status is: {len(types_df[['key', 'types']])}.\n")
if (len(types_df[['key', 'types']]) == 0) :
    print(colored.fg("green") + "\nRule 34 passed for this Open-Source Project!\n")
else :
  print(colored.fg("red") + "\nRule 34 failed for this Open-Source Project!\n")

types_df[['key', 'types']]

## **#R33 - Each issue belongs to a specific Sprint, therefore there should be a sprint identifier attached to each issue.**

In [ ]:
# THIS RULE CHECKS IF ALL ISSUES IN THE PROJECT, ARE CREATED AS PART OF A RUNNING SPRINT,
# IN THE CASE THE TEAMS WERE ORGANIZING THE WORK IN SPRINTS.

print(f'Total number of unique sprints present in this Open-Source Project is: {df.sprint.nunique()}.')
print(f'Total number of missing sprints present in this Open-Source Project is: {df.sprint.isnull().sum()}.')
print(f'Total number of unique issues present in this Open-Source Project is: {df.key.nunique()}.')
print(f'Total number of missing issues present in this Open-Source Project is: {df.key.isnull().sum()}.')

print('\nFinally, we are checking if there are issues having no issue statuses in the dataset: \n')
issue_sprint_df = df[(df['sprint'].isnull()) & pd.notnull(df["key"])]

print(len(issue_sprint_df[['key', 'sprint']]))
print(f"Total number of issues belonging to no sprints is: {len(issue_sprint_df[['key', 'sprint']])}.\n")
if (len(issue_sprint_df[['key', 'sprint']]) == 0) :
    print(colored.fg("green") + "\nRule 38 passed for this Open-Source Project!\n")
else :
  print(colored.fg("red") + "\nRule 38 failed for this Open-Source Project!\n")

#issue_sprint_df[['key', 'sprint', 'resolutiondate']]

## **#R34 - All issues must be uniquely identifiable, therefore there should be a unique identifier associated with each issue.**

In [ ]:
# THIS RULE CHECKS IF ALL ISSUES IN THE PROJECT, WITHIN THEIR CORRESPONDING SPRINTS
# HAVE THEIR UNIQUE ID-s, MAKING THEM IDENTIFYIABLE FROM OTHER ISSUES

print(f'Total number of issues present in this Open-Source Project is: {df.key.nunique()}.')
print(f'Total number of missing issues present in this Open-Source Project is: {df.key.isnull().sum()}.\n')
print(f'Total number of sprints present in this Open-Source Project is: {df.sprint.nunique()}.')
print(f'Total number of missing sprint IDs present in this Open-Source Project is: {df.sprint.isnull().sum()}.\n')

grouped_df = df.groupby("sprint")

print('\n\n*      FIRST SPRINT       *\n\n')
for key, item in grouped_df:
    print(grouped_df[['sprint', 'key']].get_group(key).to_markdown())
    if (pd.isnull(grouped_df['key'])):
      print('\n\n*        FALSEEE1        *\n\n')
      #exit()
    else:
      print('\n\n*        TRUE        *\n\n')
      print('\n\n*        NEXT SPRINT        *\n\n')
    

r39_df = df[(df['key'].isnull()) & pd.notnull(df["sprint"])]

if (len(r39_df[['key', 'sprint']]) == 0) :
  print(colored.fg("green") + "\nRule 36 passed for this Open-Source Project!\n")
else :
  print(colored.fg("red") + "\nRule 36 failed for this Open-Source Project!\n")

r39_df[['sprint', 'key']]

In [ ]:
# if pd.isnull(df['key']):
#       print('\n\n*        FALSEEE1        *\n\n')
# else:
#   print("True")

def condition(df):
  if df['key'].notna: return "text a"
  if pd.isna(df['key']): return df['key']

condition(df)

In [ ]:
new_df = df.loc[df['description'].str.startswith('The column titles of')].copy()

new_df

## **#R35 - Scrum Sprints have a starting time. There should be timestamp indicating the sprints kick-off.**

In [ ]:
# THIS RULE CHECKS IF SPRINTS WITHIN THE OPEN-SOURCE PROJECTS ARE DEFINITE IN TIME, 
# MEANING THAT EACH SPRINT MUST HAVE A STARTING TIME AS WELL AS AN ENDING TIME.

# EXPLANATION: Since not all projects have defined sprints start and end time,
# for other projects I am grouping the issues as per the sprint they were implemented in.
# After that, I am only displaying the first issue and last issue of the specific sprint.
# This way, we can see the start time of the first issue, i.e. the start time of the sprint,
# as well as the end time of the last issue from that sprint, i.e. the end time of the sprint itself.

print(f"Number of unique sprints (excluding the nan) in the Open-Source Project: {df.sprint.nunique()}\n")
print(df.sprint.unique())
print('\n\n')
#df[['sprint', 'key', 'created', 'resolutiondate']].groupby(['sprint'])
sprint_start_time_df = df[['sprint', 'key', 'created', 'resolutiondate' ]].groupby(['sprint'])
display(sprint_start_time_df.tail(1).sort_values(['sprint', 'created'], ascending=[True, True]))
# (pd.concat([g.tail(1), g.head(1)])
#    .drop_duplicates()
#    .sort_values(['sprint', 'created'], ascending=[True, True])
#    .reset_index(drop=True))

if pd.isnull(sprint_start_time_df.resolutiondate) == True:
    print(colored.fg("red") + "\nRule 40 failed for this Open-Source Project!\n")
else:
    print(colored.fg("green") + "\nRule 40 passed for this Open-Source Project!\n")


if df['sprint'].str.contains('startDate=null').any():
  print("Yep")
else:
  print("No")

In [ ]:
start_dates_count = df['sprint'].str.contains('startDate=').sum()
if start_dates_count > 0:
    print ("There are {m} sprints".format(m=start_dates_count))

In [ ]:
#df['sprint'].str.contains('startDate=')

if df['key'].isnull().any():
    print("Yep")
else:
  print("No")

In [ ]:
if df['sprint'].str.contains('startDate=').any():
  print("True")
else :
  print("False")

In [ ]:
df[['sprint', 'key', 'created', 'resolutiondate']].groupby(['sprint']).apply(display)

## **#R36 - Scrum Sprints have a completion time. There should be timestamp indicating the sprints completion/resolution.**

In [ ]:
# THIS RULE CHECKS IF SPRINTS WITHIN THE OPEN-SOURCE PROJECTS ARE DEFINITE IN TIME, 
# MEANING THAT EACH SPRINT MUST HAVE A STARTING TIME AS WELL AS AN ENDING TIME.

# EXPLANATION: Since not all projects have defined sprints start and end time,
# for other projects I am grouping the issues as per the sprint they were implemented in.
# After that, I am only displaying the first issue and last issue of the specific sprint.
# This way, we can see the start time of the first issue, i.e. the start time of the sprint,
# as well as the end time of the last issue from that sprint, i.e. the end time of the sprint itself.

print(f"Number of unique sprints (excluding the nan) in the Open-Source Project: {df.sprint.nunique()}\n")
print(df.sprint.unique())
print('\n\n')
#df[['sprint', 'key', 'created', 'resolutiondate']].groupby(['sprint'])
sprint_end_time_df = df[['sprint', 'key', 'resolutiondate']].groupby(['sprint'])
display(sprint_end_time_df.head(1).sort_values(['sprint', 'resolutiondate'], ascending=[True, True]))
# (pd.concat([g.tail(1), g.head(1)])
#    .drop_duplicates()
#    .sort_values(['sprint', 'created'], ascending=[True, True])
#    .reset_index(drop=True))
if pd.isnull(sprint_end_time_df.resolutiondate):
  print(colored.fg("red") + "\nRule 38 failed for this Open-Source Project!\n")
else:
  print(colored.fg("green") + "\nRule 38 passed for this Open-Source Project!\n")


if df['sprint'].str.contains('endDate=null').any():
  print("Yep")
else:
  print("No")

## **#R37 - There should be a minimum of one issue, representing a Sprint Backlog Item, per each Sprint.**

In [ ]:
# THIS RULE CHECKS IF ANY OF THE SPRINTS WITHIN THE OPEN-SOURCE PROJECTS
# CONTAINS NO ISSUES, i.e. NO PRODUCT BACKLOG ITEMS/SPRINT BACKLOG ITEMS 


# def get_max_items_per_sprint(g):
#     return g['key'].value_counts().idxmax() 

print(f"\n The sprint with most completed issues is: {df.groupby('sprint').count().key.idxmax()}\n")


counts = df[['key', 'sprint']].groupby(['sprint']).describe()
display(counts)
print('\n\n')

pbis_per_sprint = df.groupby(['sprint'])['key'].apply(lambda grp: list(grp.value_counts().index)).to_dict()
print(pbis_per_sprint)



# the logic of this sprint is that the sprints' length is not 0, meaning that each sprint has some issues or PBIs within. 
value = input("Check a number of PBIs that sprints might have: ")

total_pbis_per_sprint = [len(v) for v in pbis_per_sprint.values()]


# check if value of 0 exist in dict using "in" & values()
if pd.to_numeric(value) == 0:
  if pd.to_numeric(value) in total_pbis_per_sprint :
    print(f"\nYes, this project contains sprints which have {value} PBIs!")
    print(colored.fg("red") + "\nRule 42 failed for this Open-Source Project!\n")
  else: 
    print(f"\nNo, this project does not contain sprints with {value} PBIs!")
    print(colored.fg("green") + "\nRule 42 passed for this Open-Source Project!\n")
elif pd.to_numeric(value) != 0 and pd.to_numeric(value) in total_pbis_per_sprint:
  print(f"\nYes, this project contains sprints which have {value} PBIs!")  
  #print(colored.fg("green") + "\nRule 42 passed for this Open-Source Project!\n")
else:
  print(f"\nNo, this project does not contain sprints with '{value}' PBIs!")  

## **#R38 - There should be timestamp indicating the issue development kick-off.**

In [ ]:
import colored

print(f"\nTotal number of issues in the Open-Source Project are: {df['key'].count()}.")

print(f"Total number of unique issues (excluding nan) in the Open-Source Project are: {df.key.nunique()}. \n")
print('\nThese issues are as printed below:')
print(df.key.unique())


# Selecting all duplicate rows based on column: key
duplicateRowsDF = df[df.duplicated(['key'])]
print(f"\nThere are: {len(duplicateRowsDF)} issue duplicates!")

# Here I am removing all issue duplicates, while only keeping the first instance!
print('In case of duplicate issues, the duplicates will be removed! However, this step was already performed in the data cleaning part.')
df = df.drop_duplicates(subset='key', keep='first')

print(f"\nTotal number of issues (key column), after cleaning is: {df['key'].count()}\n")
print(f"{df['key'].value_counts().head(10)}\n\n")
print(f"Total number of missing values in the *key* column in the Open-Source Project: {df.key.isnull().sum()}.\n")
print(df.isnull().sum())


print('\n\nFinally, we are checking if there are issues having no start date left in the dataset: \n')
startdate_df = df[(df['created'].isnull()) & pd.notnull(df["key"])]

print(f"Total number of issues having no created date is: {len(startdate_df[['key', 'created']])}.\n")
if (len(startdate_df[['key', 'created']]) == 0) :
  print(colored.fg("green") + "\nRule 40 passed for this Open-Source Project!\n")
else :
  print(colored.fg("red") + "\nRule 40 failed for this Open-Source Project!\n")

startdate_df[['key', 'created']]

## **#R39 - There should be timestamp indicating the issue development completion.**

In [ ]:
print(f"\nTotal number of issues in the Open-Source Project are: {df['key'].count()}.")

print(f"Total number of unique issues (excluding nan) in the Open-Source Project are: {df.key.nunique()}. \n")
print('\nThese issues are as printed below:')
print(df.key.unique())


# Selecting all duplicate rows based on column: key
duplicateRowsDF = df[df.duplicated(['key'])]
print(f"\nThere are: {len(duplicateRowsDF)} issue duplicates!")


# Here I am removing all issue duplicates, while only keeping the first instance!
print('In case of duplicate issues, the duplicates will be removed! However, this step was already performed in the data cleaning part.')
df = df.drop_duplicates(subset='key', keep='first')

print(f"\nTotal number of issues (key column), after cleaning is: {df['key'].count()}\n")
print(f"{df['key'].value_counts().head(10)}\n\n")
print(f"Total number of missing values in the *key* column in the Open-Source Project: {df.key.isnull().sum()}.\n")
print(df.isnull().sum())


print('\n\nFinally, we are checking if there are issues having no end date left in the dataset: \n')
enddate_df = df[(df['resolutiondate'].isnull()) & pd.notnull(df["key"])]

print(f"Total number of issues having no end date is: {len(enddate_df[['key', 'resolutiondate']])}.\n")
if (len(enddate_df[['key', 'created']]) == 0) :
    print(colored.fg("green") + "\nRule 41 passed for this Open-Source Project!\n")
else :
  print(colored.fg("red") + "\nRule 41 failed for this Open-Source Project!\n")
enddate_df[['key', 'resolutiondate', 'sprint']]